# FESDModel

FESD - Fault estimation for skeleton detection - is a suite that aims at finding faults in joints of skeletons, which are detected by human pose estimatiors.

FESDData is the sister project to this notebook, which aims at recording depth and rgb data, as well as populating the data with human poses from variing human pose estimators.

Furthermore, FESTData augments all data based on joint confidence.

FFESDModel aims to develop and evaluate a model based on the faulty and augmented joint data as well as RGBD data.

## Libraries

We need a range of libraries which are imported here. We also define some constants.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from pathlib import Path
from time import time
import datetime

import torch
from tqdm.notebook import tqdm

import json
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None

from model import FESD, FESDv2, train, val, test

from utils.mode import Mode
from utils import get_model_iter, get_model_iter_all

from data import Frame


num_gpus = torch.cuda.device_count()
is_cuda = torch.cuda.is_available()
print(f"Num cuda GPUs: {num_gpus}")

Num cuda GPUs: 0


## Model

Build the model according to the chosen mode

### Train Model

In the following we define the training function and train a network on the training data.

In [3]:
batchsize = 40
im_size = 64
epochs = 100

# gradient clipping margin
clip = 0.5

test_exercises = ['E-0.01', 'E-1.01', 'E-2.01', 'E-3.01']

all_modes = True
use_v2 = True

CE = torch.nn.CrossEntropyLoss()
if is_cuda:
    CE = CE.cuda()

model_iterator = []
if all_modes:
    model_iterator = get_model_iter_all(is_cuda, use_v2, test_exercises, epochs, batchsize, im_size, clip)
else:
    model_iterator = get_model_iter(mode, is_cuda, use_v2, test_exercises, epochs, batchsize, im_size, clip)

In [4]:
model_columns = ["epoch", "iteration", "joint_id",
                  "gts", "preds", "confidences", 
                  "Avg loss", "loss", "accuracy", 
                  "tp", "tn", "fp", "fn", "precision", "recall", "f1", 
                  "cohens_kappa", "learning_rate",
                  "train_test", "exercise", "simplified", "mode", "use_v2"]
                  
df_model = pd.DataFrame(columns=model_columns)
pb = tqdm(range(1, epochs + 1), desc='Epoch')

for epoch in pb:
    print(f"--- {epoch:3d} ---")
    for mode, model, optimizer, scheduler, train_loader, _ in model_iterator:
        tic = time()
        torch.cuda.empty_cache()
        
        loss = train(train_loader, model, optimizer, CE, scheduler, clip, epoch, epochs, is_cuda, mode, df_model, use_v2)
        
        crit_1 = df_model["epoch"] == epoch
        crit_2 = df_model["mode"] == mode.name.lower()
        last_row = df_model[crit_1 & crit_2].mean(numeric_only=True)
        last_row["p"] = last_row["tp"] + last_row["fp"]
        last_row["n"] = last_row["tn"] + last_row["fn"]
        last_row["positives"] = last_row["p"] / (last_row["n"] + last_row["p"])
        pb.set_description(f'Epoch (mode: {mode.name.lower().replace("_", " "):>10}, lr: {optimizer.param_groups[0]["lr"]:.5f}, loss: {last_row["Avg loss"]:.3f})')
        
        print(f'Epoch (mode: {mode.name.lower().replace("_", " "):>10}, lr: {optimizer.param_groups[0]["lr"]:.3f}, loss: {last_row["Avg loss"]:.5f}, acc: {last_row["accuracy"]:.3f}, f1: {last_row["f1"]:.3f}, precision: {last_row["precision"]:.3f}, recall: {last_row["recall"]:.3f}, kappa: {last_row["cohens_kappa"]:.3f}, p/(p + n): {last_row["positives"]:.3f}, time: {time() - tic:.2f}s)')      

        if (epoch) % 10 == 0:
            torch.save(model.state_dict(), os.path.join(Path('checkpoints'), f"{mode.name.lower()}_{epoch}_ckpt.pth")) 
    
for mode, model, _, _, _, _ in model_iterator:
    torch.save(model.state_dict(), os.path.join(Path('checkpoints'), f"{mode.name.lower()}_last_ckpt.pth")) 
    print(f"model saved {os.path.join(Path('checkpoints'), f'last_ckpt.pth')}!")

df_model.to_parquet('ModelAnalysis.parquet.gzip', compression='gzip') 

Epoch:   0%|          | 0/100 [00:00<?, ?it/s]

---   1 ---
Mode.LIMBS


### Test Model

In [ ]:
for mode, model, _, _, _, test_loader in tqdm(model_iterator):
  model.eval()
  test(test_loader, model, CE, is_cuda, mode, df_model, use_v2)

df_model.to_parquet('ModelAnalysis.parquet.gzip', compression='gzip') 

  0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
df_model.to_parquet('ModelAnalysis.parquet.gzip', compression='gzip') 